upgrade fastai and update sentencepiece

In [2]:
!pip install fastai -q --upgrade
!pip install sentencepiece!=0.1.90,!=0.1.91

     |████████████████████████████████| 194kB 7.3MB/s 
     |████████████████████████████████| 12.8MB 261kB/s 
     |████████████████████████████████| 61kB 8.8MB/s 
     |████████████████████████████████| 776.8MB 23kB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.
     |████████████████████████████████| 1.2MB 8.3MB/s 


In [3]:
from fastai.text.all import *
from fastai import *
import pandas as pd
import numpy as np
import torch as torch

ModuleNotFoundError: ignored

In [4]:
# mount colab
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# path contains Train.csv and Test.csv files
path = Path('/content/drive/MyDrive/ml/projects/sa')

# path for fastai dataloaders
path_dls = Path('/content/drive/MyDrive/ml/projects/sa/dls')

# path for save sentencepiece model and vocab
spm_path = path/'models/spms3'

# path to save models
model_path = path/'models'

In [7]:
lang='ar'
# batch size
bs=128

In [8]:
# setting random seed
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    random.seed(seed_value) # Python
    if use_cuda: 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False


random_seed(42,True)

In [9]:
train_df = pd.read_csv(path/'Train.csv')

In [10]:
test_df = pd.read_csv(path/'Test.csv')

# Training a language model on Train and Test set

In [25]:
random_seed(42,True)

In [26]:
# concatenate train and test dataframe
text_df = pd.concat([train_df, test_df], sort=False)

In [27]:
# a sentencepiece tokenizer with a vocab_sz of 300
tok = SentencePieceTokenizer(lang=lang, cache_dir=spm_path, vocab_sz=300)

# dataloader for language model
dls_fn = DataBlock(blocks=TextBlock.from_df('text', is_lm=True, tok=tok),
    splitter=RandomSplitter(0.05, seed=42),
    get_x=ColReader('text')
).dataloaders(text_df, bs=bs)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [30]:
len(dls_fn.vocab)

344

In [28]:
# save dataloader for later use
torch.save(dls_fn, path/'dls/dls_fn_300.pkl')

In [29]:
# initialize a language model
learn = language_model_learner(dls_fn, AWD_LSTM, pretrained=False, drop_mult=1., wd=0.1,
                               metrics=[accuracy, Perplexity()]).to_fp16()

In [30]:
learn.path = model_path

In [32]:
# training
learn.fit_one_cycle(30, 3e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.492737,4.286273,0.119003,72.695030,03:08
1,3.729388,3.571048,0.230364,35.553825,03:08
2,3.541823,3.430597,0.251037,30.895084,03:05
3,3.489334,3.390949,0.254104,29.694126,03:04
4,3.496766,3.391620,0.254275,29.714041,03:02
5,3.492847,3.389706,0.255441,29.657228,03:00
6,3.468917,3.377698,0.254629,29.303236,03:00
7,3.434980,3.331117,0.262943,27.969578,02:59
8,3.398905,3.315434,0.266305,27.534328,03:00
9,3.372394,3.277642,0.271239,26.513182,03:02


In [34]:
lm_ft_pr = [model_path/f'{lang}_ft', model_path/f'{lang}_ft_vocab']

In [35]:
# save model
learn.save(lm_ft_pr[0])

Path('/content/drive/MyDrive/ml/projects/sa/models/ar_ft.pth')

In [36]:
# save vocab
with open(lm_ft_pr[1], 'wb') as f:
      pickle.dump(learn.dls.vocab, f)

In [37]:
# save encoder 
learn.save_encoder(model_path/'ft_encoder_300')

# Training classifier

In [ ]:
random_seed(42,True)

In [ ]:
# dls_fn = torch.load(path/'dls/dls_fn_300.pkl')

## Create dataloaders

In [38]:
# sentencepiece tokenizer with spm model trained in pretraining step
tok = SentencePieceTokenizer(lang=lang, sp_model=f'{spm_path}/spm.model', vocab_sz=300)

# training dataloader
train_dls = DataBlock(
    blocks=(TextBlock.from_df('text', tok=tok, vocab=dls_fn.vocab), CategoryBlock),
    splitter = RandomSplitter(0.2, seed=42),
    get_x=ColReader('text'), 
    get_y=ColReader('label')
    ).dataloaders(train_df, bs=bs)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [39]:
torch.save(train_dls, path/'train_dls_lstm300_test.pkl')

In [11]:
# train_dls = torch.load(path/'train_dls_lstm300_test.pkl')

In [12]:
train_dls.show_batch(max_n=4)

,text,category
0,▁xxbos ▁p ar ol es ▁wa dha 3 ▁fi h ▁t et s d am ▁ kh ay f ▁t et r d m ▁ mou s h ▁ena ▁li ▁n 9 oul l ▁7 a 9 ▁w ▁ki ma ▁g hi ri ▁n et r dem ▁n cha re f ▁ta s h ki la ▁w ▁7 w om ▁ eb da 3 at ▁b l ▁k am ▁d ra p e a u wa t ▁n h ez ▁w ▁ken ▁l za z ▁k ha la na ▁f l ▁h am ▁sa ra 7 ▁ na h ki ▁ta la 3 ▁ ha b t ▁dima ▁ kh ay dh ▁1 0 0 ▁m n ▁ kh o t ta ▁f y ▁ra ssi ▁3 al ▁3 wa j ▁9 al bi ▁fa y dh ▁t ay r ▁w n t y ▁b ay dh ▁ma b rou,-1
1,▁xxbos ▁el ▁ i 3 lem ▁m te 3 na ▁w ▁la ▁7 at ta ▁7 ad ▁g hi rou ▁5 at er ▁fe m ma ▁fi ▁3 a h d ▁el ▁be ji ▁chkoun ▁7 aw el ▁ em a ▁wa 9 tha ▁7 aw lou ▁ya 3 m lou ▁3 rou ch y ya ▁l 9 aw ou ha ▁7 k eya ▁f er g ha ▁d on c ▁ raj 3 ou ▁e j j me 3 a ▁m te 3 ▁el ▁f et na ▁h ed hou ma ▁l el ▁ i 3 lem ▁a ya ▁si di ▁b de w ▁b ▁n ed ya ▁el ▁fe ni ▁3 am lou ▁men ha ▁m ch k la ▁w ▁ba 3 ed ▁nes s ma ▁w ▁ba 3 ed ▁fa c ▁man nou ba ▁w ▁ba 3 ed ▁ ez ay tou na ▁w ▁7 k e,-1
2,▁xxbos ▁sa m hi ni ▁ya ▁ba i a ▁kif ▁n ek t eb ▁ca v a ▁kif ▁ na b 3 et na l 9 a ▁barcha ▁a 8 la t ▁li ▁ i ▁h â ta ▁el ka v e ▁fi ▁en g li s h ▁ te 3 a on ▁ i ▁fi ▁el ▁k et i ba ▁ama ▁kif ▁n ez r eb ▁ na b 3 et ▁ye b an ▁eli ▁8 al et ▁ o k ▁ ni o ta ▁ ha bi t ▁n ek t b la k ▁n et la m o ▁wa ▁kol ▁ i om ▁ ek t b o ▁il a ▁3 ala ▁a j ma 3 in ▁ na b de o ▁a h na ▁ya ▁ba i a ▁ta o a ▁el ▁jam i 3 ▁ye o as lo ▁ma 3 a na ▁ ar jou,-1
3,▁xxbos ▁s é lem ▁ al ay kom ▁brabi ▁en i ▁mo wa t é n ▁men ▁s ou ssa ▁m ch it ▁3 am ▁8 2 ▁li t ali a ▁m 3 a ▁mar ti ▁b ▁p as p or i ▁b ▁kol ▁chay ▁w ▁sa h é l ▁rabi ▁w ▁ ji b t ▁ 2 ▁w l é d ▁l oul ▁ ou sse ma ▁ ab d el k ad er ▁w ▁ 2 ▁bech ir ▁ ab d el k ad er ▁w ▁k on t ▁fi ▁ it ali a ▁n ek h dem ▁ba 7 ar ▁n ha r ▁men ▁n ha ra t ▁5 raj t ▁b ek ri ▁n ek h di m ▁w ▁5 ali t ▁mar ti ▁m 3 a ▁s 8 ar ▁fi ▁d ar ▁mar ti ▁5 ar ji t ▁s be h ▁bi ch ▁ta 9,0


## Create learner

In [35]:
learn_tr = text_classifier_learner(train_dls, AWD_LSTM, metrics=[accuracy], drop_mult=0.8,
                                    pretrained=False).to_fp16()
learn_tr.load_encoder(model_path/'ft_encoder_300')
learn_tr.freeze()

## Training using gradual unfreezing

In [26]:
learn_tr.fit_one_cycle(3, 5e-2, moms=(0.8, 0.7, 0.8))

epoch,train_loss,valid_loss,accuracy,time
0,0.736462,0.673014,0.769786,02:14
1,0.726154,0.656314,0.785929,02:14
2,0.695689,0.640963,0.788071,02:14


In [27]:
learn_tr.save(path/'models/temp0')

Path('/content/drive/MyDrive/ml/projects/sa/models/temp0.pth')

In [22]:
# learn_tr.load(path/'models/temp0')

In [23]:
learn_tr.freeze_to(-2)
learn_tr.fit_one_cycle(3, slice(2e-2/(2.6**4),2e-2), moms=(0.8, 0.7, 0.8))

epoch,train_loss,valid_loss,accuracy,time
0,0.662954,0.599412,0.820429,01:19
1,0.634857,0.576196,0.832000,01:19
2,0.580597,0.574900,0.835857,01:19


In [24]:
learn_tr.save(path/'models/temp2')

Path('/content/drive/MyDrive/ml/projects/sa/models/temp2.pth')

In [32]:
# learn_tr.load(path/'models/temp2')

In [33]:
learn_tr.freeze_to(-3)
learn_tr.fit_one_cycle(2, slice(7e-3/(2.6**4),7e-3), moms=(0.8, 0.7, 0.8))

epoch,train_loss,valid_loss,accuracy,time
0,0.590876,0.574029,0.836071,01:19
1,0.554038,0.582437,0.839143,01:20


In [16]:
learn_tr.save(path/'models/temp3')

Path('/content/drive/MyDrive/ml/projects/sa/models/temp3.pth')

In [36]:
# learn_tr.load(path/'models/temp3')

In [37]:
learn_tr.unfreeze()
learn_tr.fit_one_cycle(4, slice(3e-3/(2.6**4),3e-3), moms=(0.8, 0.7, 0.8))

epoch,train_loss,valid_loss,accuracy,time
0,0.303624,0.450897,0.838571,01:21
1,0.289323,0.474417,0.837929,01:21
2,0.260646,0.479603,0.838714,01:21
3,0.241612,0.507132,0.839786,01:21


In [126]:
learn_tr.save(path/'models/subword_classifier350')

Path('/content/drive/MyDrive/ml/projects/sa/models/subword_classifier350.pth')

In [ ]:
# learn_tr.load(path/'models/subword_classifier300')

#inference

In [27]:
subm = pd.read_csv(path/'SampleSubmission.csv')

In [28]:
# create a test dataloader
test_dl=train_dls.test_dl(test_df, text_col='text')

In [29]:
# get predictions
preds = learn_tr.get_preds(dl=test_dl)

In [30]:
subm['ID'] = test_df['ID']
subm['label'] = torch.argmax(preds[0], dim=1) - 1
subm.set_index('ID', inplace=True)

In [31]:
# save submission
subm.to_csv(path/'subm7.csv')